In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from glob import glob
import shutil

In [2]:
data_dir = os.getcwd()
img_dir = os.path.join(data_dir, 'images/')
excel_path = os.path.join(data_dir, 'hh_video_data.xlsx')
label_path = os.path.join(data_dir, 'handhygiene_label.csv') 
df_ex = pd.read_excel(excel_path)

In [3]:
def move_target_data(img_dir, target_str):
    nonetarget_dir = os.path.join(img_dir, target_str)
    rest_img_dir = os.path.join(img_dir, '*.jpg')
    !mv $rest_img_dir $nonetarget_dir

In [4]:
def add_to_data_dict(video_id, img_date, img_name, clip_num):
    hh_dict['video_id'].append(video_id)
    hh_dict['date'].append(img_date)
    hh_dict['img_name'].append(img_name+'.jpg')
    hh_dict['clip_num'].append(clip_num)
    return 

In [8]:
hh_dict = {'video_id':[], 'date':[], 'img_name':[], 'clip_num':[]}
#target_str = 'clean'
target_dir = os.path.join(img_dir, 'clean')
img_total = glob(os.path.join(img_dir, '*.jpg'))
img_total.sort()
cnt = 0
    
for image in tqdm(img_total):
    img_name= image.split('/')[-1].split('.')[0]
    video_id = int(img_name.split('_')[0])
    img_date = img_name.split('_')[1]
    img_frame_num = img_name[-6:] # ex: 000001
        

    target_clips = df_ex[df_ex['video_id'] == video_id]['target_frame'].iloc[0]
                                                                                
    #target_str = 'clean'
    targets = {'clean': target_clips}
    if targets['clean'] is np.nan:
        continue
        
        
    clips = targets['clean'].strip().split(',')
        
    for i, el in enumerate(clips):
        clip_num = i
        start = el.strip().split('-')[0]
        end = el.strip().split('-')[1]
        
        #print(i, start, end)
        # if current frame is in range of (start, end)
        if int(img_frame_num) in range(int(start), int(end)+1):
            # move data to target directory
            shutil.move(image, target_dir)
            cnt += 1
            add_to_data_dict(video_id, img_date, img_name, clip_num)
            break
    
print("{} images moved to {}".format(cnt, target_dir))
move_target_data(img_dir, 'notclean')


100%|██████████| 3402/3402 [00:01<00:00, 1882.07it/s]

1010 images moved to /data/handhygiene/images/clean


In [13]:
df = pd.DataFrame(hh_dict)
df[df['clip_num'] == 5]

,clip_num,date,img_name,video_id
358,5,20180806,1_20180806_frames001191.jpg.jpg,1
359,5,20180806,1_20180806_frames001192.jpg.jpg,1
360,5,20180806,1_20180806_frames001193.jpg.jpg,1
361,5,20180806,1_20180806_frames001194.jpg.jpg,1
362,5,20180806,1_20180806_frames001195.jpg.jpg,1
363,5,20180806,1_20180806_frames001196.jpg.jpg,1
364,5,20180806,1_20180806_frames001197.jpg.jpg,1
365,5,20180806,1_20180806_frames001198.jpg.jpg,1
366,5,20180806,1_20180806_frames001199.jpg.jpg,1
367,5,20180806,1_20180806_frames001200.jpg.jpg,1
